In [1]:
## Import Library ##

import os
import random
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from keras.models import Model
from keras import optimizers, applications
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from keras.preprocessing.image import ImageDataGenerator

In [2]:
## Seed ##
from tensorflow import set_random_seed
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    set_random_seed(0)
seed_everything()

In [3]:
## Configuration ##
HEIGHT = 512
WIDTH = 512
CANAL = 3
N_CLASSES = 5

In [4]:
## Load&Preprocess data ##
test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
test["id_code"] = test["id_code"].apply(lambda x: x + ".png")

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(  
        dataframe=test,
        directory = "../input/aptos2019-blindness-detection/test_images/",
        x_col="id_code",
        target_size=(HEIGHT, WIDTH),
        batch_size=1,
        shuffle=False,
        class_mode=None)

In [6]:
## Model ##
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = InceptionResNetV2(weights = None, include_top = False, input_tensor = input_tensor)
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='softmax', name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

In [7]:
## Eval ##
test_generator.reset()
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

model = create_model(input_shape=(HEIGHT, WIDTH, CANAL), n_out=N_CLASSES)
model.load_weights('../input/d/oriori54/aptos-tf-pretrained-models/checkpoint_inceptionresnet.ckpt')

preds = model.predict_generator(test_generator, steps=STEP_SIZE_TEST)
predictions = [np.argmax(pred) for pred in preds]

In [8]:
## Submission File ##
filenames = test_generator.filenames
results = pd.DataFrame({'id_code':filenames, 'diagnosis':predictions})
results['id_code'] = results['id_code'].map(lambda x: str(x)[:-4])
results.to_csv('submission.csv',index=False)
results.head(10)